In [17]:
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime
from geopy.distance import geodesic
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from bs4 import BeautifulSoup

# ====== 設定ここから ======
HTML_DIR = 'house_pricing/html_files'  # ← ここにテスト用htmlファイルを保存！

import time
TIMESTAMP = time.strftime('%Y-%m-%d %H:%M:%S')
OUTPUT_CSV = './scraped_mansion_data'+ TIMESTAMP +'.csv'
# ====== 設定ここまで ======

property_data = []

def extract_property_info(property_soup):
    """1物件分の情報を複数部屋分すべて抽出"""
    property_info_list = []

    try:
        # 基本情報の抽出（物件名・住所・交通・築年数）
        property_name = property_soup.select_one('.property-detail-content__head-title a')
        property_name = property_name.get_text(strip=True) if property_name else ''

        address = access = age = ''
        info_table = property_soup.find('table', class_='property-detail-content_main')
        if info_table:
            for row in info_table.find_all('tr'):
                th = row.find('th')
                td = row.find('td')
                if not th or not td:
                    continue
                key = th.get_text(strip=True)
                val = td.get_text(separator=' ', strip=True)
                if key == '住所':
                    address = val
                elif key == '交通':
                    access = val
                elif key == '築年数':
                    age = val

        # 各部屋の賃貸情報を取得
        recommend_tables = property_soup.find_all('table', class_='recommendTable')
        for table in recommend_tables:
            for tbody in table.find_all('tbody', class_='recommend_row'):
                tr = tbody.find('tr')
                if not tr:
                    continue
                tds = tr.find_all('td')
                if len(tds) < 9:
                    continue

                image_tag = tds[0].find('img')
                image_url = image_tag.get('data-original') if image_tag else ''

                room_name = tds[1].get_text(strip=True)
                rent = tds[2].get_text(strip=True)
                deposit = tds[3].get_text(strip=True)
                key_money = tds[4].get_text(strip=True)
                area = tds[5].get_text(strip=True)
                floor_plan = tds[6].get_text(strip=True)
                floor = tds[7].get_text(strip=True)
                facing = tds[8].get_text(strip=True)

                property_info_list.append({
                    '物件名': property_name,
                    '住所': address,
                    '交通': access,
                    '築年数': age,
                    '部屋名': room_name,
                    '賃料': rent,
                    '敷金': deposit,
                    '礼金': key_money,
                    '占有面積': area,
                    '間取り': floor_plan,
                    '所在階': floor,
                    '向き': facing,
                    '画像': image_url
                })
    except Exception as e:
        print(f"Error extracting property info: {e}")

    return property_info_list


def scrape_html_files(html_dir):
    """指定ディレクトリ内のhtmlファイルをすべて処理し、複数部屋分のデータを抽出"""
    all_properties = []

    for filename in os.listdir(html_dir):
        if filename.endswith('.html'):
            path = os.path.join(html_dir, filename)
            with open(path, 'r', encoding='utf-8') as f:
                soup = BeautifulSoup(f, 'html.parser')
                properties = soup.select('section.chart_list_layout')
                for prop in properties:
                    prop_infos = extract_property_info(prop)
                    all_properties.extend(prop_infos)
    
    return pd.DataFrame(all_properties)

#現状はあくまで予測対象のデータのみを抽出するロジックになっているが、少し加工（dfと学習用データのパスを分けるなど）すれば、
#学習用データのDataframeを作成することも可能である。

if __name__ == '__main__':
    scraped_df = scrape_html_files(HTML_DIR)
    scraped_df['id'] = scraped_df.index + 1
    scraped_df.to_csv(OUTPUT_CSV, index=False)
    print(f"\nDone! {len(scraped_df)} properties scraped and saved to {OUTPUT_CSV}")

# =====================
# 関数定義（既存処理 + 追加処理）
# =====================

def clean_rent(rent_str):
    if pd.isna(rent_str): return np.nan
    rent_cleaned = re.sub(r'\(.*?\)', '', rent_str).replace(',', '').replace('万円', '')
    try: return float(rent_cleaned)
    except: return np.nan

def process_deposit_fee(fee_str, rent_value):
    if pd.isna(fee_str) or fee_str.strip() == '':
        return 0.0
    try:
        if 'ヶ月' in fee_str:
            return float(fee_str.replace('ヶ月', '').replace(',', '').strip())
        elif '円' in fee_str and rent_value and not np.isnan(rent_value):
            yen_amount = int(fee_str.replace('円', '').replace(',', '').strip())
            return yen_amount / (rent_value * 10000)
    except:
        return 0.0
    return 0.0

def calculate_elapsed_months(built_year_str):
    if pd.isna(built_year_str): return 0
    built_year_str = built_year_str.replace('年', '/').replace('月', '').replace('築', '').replace('新築', '2025/04')
    try:
        built_date = datetime.strptime(built_year_str, "%Y/%m")
        today = datetime(2025, 4, 1)
        return (today.year - built_date.year) * 12 + (today.month - built_date.month)
    except: return 0

def clean_area(area_str):
    if pd.isna(area_str): return 0
    area_str = area_str.replace('m²', '').replace('㎡', '').replace(',', '').strip()
    try: return float(area_str)
    except: return 0

def extract_ward(address):
    if pd.isna(address): return None
    match = re.search(r'東京都(.{2,3}区)', address)
    if match: return match.group(1)
    return None

wards_centers = {
    '千代田区': (35.694003, 139.753595), '中央区': (35.670651, 139.771861), '港区': (35.658068, 139.751599),
    '新宿区': (35.693840, 139.703549), '文京区': (35.708068, 139.752167), '台東区': (35.712607, 139.779996),
    '墨田区': (35.710722, 139.801497), '江東区': (35.672854, 139.817410), '品川区': (35.609226, 139.730186),
    '目黒区': (35.641463, 139.698171), '大田区': (35.561257, 139.716051), '世田谷区': (35.646572, 139.653247),
    '渋谷区': (35.661777, 139.704051), '中野区': (35.707399, 139.663835), '杉並区': (35.699566, 139.636438),
    '豊島区': (35.726118, 139.716605), '北区': (35.752804, 139.733481), '荒川区': (35.736080, 139.783369),
    '板橋区': (35.751165, 139.709244), '練馬区': (35.735623, 139.651658), '足立区': (35.775664, 139.804479),
    '葛飾区': (35.743575, 139.847180), '江戸川区': (35.706657, 139.868427)
}

def calculate_center_distance(row):
    ward = row['区名']
    if ward not in wards_centers: return 0
    ward_center = wards_centers[ward]
    distances = [
        geodesic(ward_center, wards_centers['千代田区']).meters,
        geodesic(ward_center, wards_centers['中央区']).meters,
        geodesic(ward_center, wards_centers['港区']).meters
    ]
    return min(distances)

def process_floor_range(floor_str):
    if pd.isna(floor_str): return '0-4階'
    match = re.search(r'(\d+)', floor_str)
    if not match: return '0-4階'
    floor_num = int(match.group(1))
    lower = (floor_num // 5) * 5
    return f"{lower}-{lower+4}階"

def extract_station(text):
    if pd.isna(text): return np.nan
    match = re.search(r'(\S+?)駅', text)
    if match: return match.group(1)
    return np.nan

def extract_walk_time(text):
    if pd.isna(text):
        return np.nan
    # 「徒歩」から「分」までの間に任意の空白や全角数字があるパターンに対応
    match = re.search(r'徒歩\s*([0-9０-９]+)\s*分', text)
    if match:
        # 全角数字 → 半角に変換
        digits = match.group(1).translate(str.maketrans('０１２３４５６７８９', '0123456789'))
        return int(digits)
    return np.nan


# =====================
# データ読み込みと前処理
# =====================


# --- 前処理適用 ---
scraped_df['賃料_cleaned'] = scraped_df['賃料'].apply(clean_rent).fillna(0)
scraped_df['敷金_cleaned'] = scraped_df.apply(lambda x: process_deposit_fee(x['敷金'], x['賃料_cleaned']), axis=1).fillna(0)
scraped_df['礼金_cleaned'] = scraped_df.apply(lambda x: process_deposit_fee(x['礼金'], x['賃料_cleaned']), axis=1).fillna(0)
scraped_df['経過月数'] = scraped_df['築年数'].apply(calculate_elapsed_months).fillna(0)
scraped_df['占有面積_cleaned'] = scraped_df['占有面積'].apply(clean_area).fillna(0)
scraped_df['区名'] = scraped_df['住所'].apply(extract_ward).fillna('その他')
scraped_df['都心3区最短距離(m)'] = scraped_df.apply(calculate_center_distance, axis=1).fillna(0)
scraped_df['最寄り駅名'] = scraped_df['交通'].apply(extract_station).fillna('その他')
scraped_df['徒歩分数'] = scraped_df['交通'].apply(extract_walk_time).fillna(0)
scraped_df['所在階カテゴリ'] = scraped_df['所在階'].apply(process_floor_range)
scraped_df['向き'] = scraped_df['向き'].fillna('不明')


train_data = pd.read_csv("house_pricing/Arakawa_scraped_mansion_data.csv")

# 前処理
train_data['賃料_cleaned'] = train_data['賃料'].apply(clean_rent).fillna(0)
train_data['敷金_cleaned'] = train_data.apply(lambda x: process_deposit_fee(x['敷金'], x['賃料_cleaned']), axis=1).fillna(0)
train_data['礼金_cleaned'] = train_data.apply(lambda x: process_deposit_fee(x['礼金'], x['賃料_cleaned']), axis=1).fillna(0)
train_data['占有面積_cleaned'] = train_data['占有面積'].apply(clean_area).fillna(0)
train_data['経過月数'] = train_data['築年数'].apply(calculate_elapsed_months).fillna(0)
train_data['区名'] = train_data['住所'].apply(extract_ward).fillna('その他')
train_data['都心3区最短距離(m)'] = train_data.apply(calculate_center_distance, axis=1).fillna(0)
train_data['最寄り駅名'] = train_data['交通'].apply(extract_station).fillna('その他')
train_data['徒歩分数'] = train_data['交通'].apply(extract_walk_time).fillna(0)
train_data['所在階カテゴリ'] = train_data['所在階'].apply(process_floor_range)
train_data['向き'] = train_data['向き'].fillna('不明')

# =====================
# 特徴量・目的変数
# =====================

numeric_features = ['経過月数', '礼金_cleaned', '敷金_cleaned', '占有面積_cleaned', '都心3区最短距離(m)','徒歩分数']
#categorical_features = ['所在階カテゴリ', '向き']
target = '賃料_cleaned'

X = train_data[numeric_features]
y = train_data[target]

# =====================
# 多重共線性チェック
# =====================

print("\n【VIF（分散拡大係数）】")
vif_data = pd.DataFrame()
vif_data["feature"] = numeric_features
vif_data["VIF"] = [variance_inflation_factor(train_data[numeric_features].values, i) for i in range(len(numeric_features))]
print(vif_data)

# =====================
# モデルトレーニング
# =====================

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features)
#  ,('cat', OneHotEncoder(drop='first'), categorical_features)
])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#model.fit(X_train, y_train)

# 不要な特徴量を除外（向きと階数カテゴリを削除）
#drop_columns = [col for col in X.columns if '向き_' in col or '所在階カテゴリ_' in col]
#X_reduced = X.drop(columns=drop_columns)

# 学習再実行
model.fit(X, y)

# 精度評価
#from sklearn.metrics import r2_score

X_new = scraped_df[numeric_features]
y_new = scraped_df[target]

y_pred = model.predict(X_new)


print("\n【モデル精度】")
print("R^2（決定係数）:", model.score(X_new, y_new))

#違う区の家賃を正しく予想するには至らない、、
#同じ区でやれば、おそらくもっと精度が出る、、

# 各特徴量の重要度
# 学習後に LinearRegression モデルの係数にアクセス
regressor = model.named_steps['regressor']  # Pipeline内のLinearRegressionにアクセス
coefficients = regressor.coef_

# 特徴量名を取得（OneHotEncoder含めてすべての変換後の名前）
# カテゴリ特徴量のエンコーダーの列名を取得
#ohe = model.named_steps['preprocessor'].named_transformers_['cat']
#ohe_feature_names = ohe.get_feature_names_out(categorical_features)

# 全特徴量名（数値特徴量 + OneHotEncoderで展開されたカテゴリ特徴量）
all_feature_names = numeric_features  #+ list(ohe_feature_names)

# DataFrameで表示
importance_df = pd.DataFrame({
    '特徴量': all_feature_names,
    '係数': coefficients
}).sort_values(by='係数', ascending=False)

print(importance_df)


# (4) 予測


scraped_df = scraped_df.reset_index(drop=True)

result = pd.DataFrame({
    'id': scraped_df['id'],
    '区名': scraped_df['区名'],
    '最寄り駅名': scraped_df['最寄り駅名'],
    '徒歩分数': scraped_df['徒歩分数'],
    '経過月数': scraped_df['経過月数'],
    '物件名': scraped_df['物件名'],
    '予測賃料': y_pred,
    '実際の賃料': scraped_df['賃料_cleaned']
})

print(result)

result.to_csv('house_prediction_result_' + TIMESTAMP + '.csv')






Done! 269 properties scraped and saved to ./scraped_mansion_data2025-05-12 14:26:15.csv

【VIF（分散拡大係数）】
        feature        VIF
0          経過月数   1.394322
1    礼金_cleaned   1.469094
2    敷金_cleaned   1.194564
3  占有面積_cleaned   1.040044
4   都心3区最短距離(m)  12.873039
5          徒歩分数   1.207427

【モデル精度】
R^2（決定係数）: 0.7084469362309865
            特徴量        係数
3  占有面積_cleaned  4.751530
2    敷金_cleaned  0.132942
4   都心3区最短距離(m)  0.000000
5          徒歩分数 -0.386112
1    礼金_cleaned -1.214584
0          経過月数 -1.228177
      id   区名   最寄り駅名  徒歩分数  経過月数              物件名       予測賃料  実際の賃料
0      1  荒川区     宮ノ前   1.0   355             エイビル  14.649743   11.0
1      2  荒川区     熊野前  12.0   456            レジナス椿   3.981404    6.7
2      3  荒川区     三ノ輪   6.0   374          プリモネージュ  12.904641    8.7
3      4  荒川区     三ノ輪   4.0    87           南千住ハウス   9.137248    7.2
4      5  荒川区     熊野前   4.0    73            ポルックス  10.426803    8.0
..   ...  ...     ...   ...   ...              ...        ...    ...
264

In [2]:
! ls 

 README.ipynb
 anaconda_projects
 dance_circle
 equity_IR_analysis
'house_prediction_result_2025-05-11 15:35:47.csv'
'house_prediction_result_2025-05-12 12:04:49.csv'
 house_pricing
 koutu_honda_final
 koutuu_honda
 openwork_rep_classification
'scraped_mansion_data2025-05-11 15:34:20.csv'
'scraped_mansion_data2025-05-11 15:35:47.csv'
'scraped_mansion_data2025-05-12 12:04:49.csv'


In [3]:
! ls house_pricing/html_files

676_13.html  676_14.html  676_15.html  676_16.html  escape


In [4]:
print(scraped_df)

                 物件名                  住所                        交通       築年数  \
0               エイビル   東京都荒川区西尾久2丁目4番17号         都電荒川線 宮ノ前駅 徒歩 1 分   1995年9月   
1              レジナス椿         東京都荒川区町屋5丁目   日暮里・舎人ライナー 熊野前駅 徒歩 12 分   1987年4月   
2            プリモネージュ        東京都荒川区南千住2丁目     東京メトロ日比谷線 三ノ輪駅 徒歩 6 分   1994年2月   
3             南千住ハウス   東京都荒川区南千住5丁目26番9号     東京メトロ日比谷線 三ノ輪駅 徒歩 4 分   2018年1月   
4              ポルックス     東京都荒川区東尾久5丁目6-3    日暮里・舎人ライナー 熊野前駅 徒歩 4 分   2019年3月   
..               ...                 ...                       ...       ...   
264            メイフラワ         東京都荒川区町屋4丁目                             1991年3月   
265  レオパレスフラワーガーデン町屋         東京都荒川区町屋6丁目          京成本線 町屋駅 徒歩 16 分   2012年4月   
266          レジデンス久野  東京都荒川区東尾久3丁目11番15号      都電荒川線 東尾久三丁目駅 徒歩 4 分   1999年1月   
267     Glanz南千住EAST   東京都荒川区南千住5丁目14番9号  JR常磐線(上野～取手) 南千住駅 徒歩 6 分  2019年12月   
268        PARKHILLS         東京都荒川区荒川2丁目       都電荒川線 荒川二丁目駅 徒歩 4 分  2015年12月   

                     部屋名             賃料